<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Preprocessing/ColomboRestaurantsPreprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
# Load the uploaded file
file_path = "/content/drive/My Drive/DataPre/preprocessed_user_inputs.csv"
df = pd.read_csv(file_path)

# Generate synthetic 'maximum_distance' column with values ranging from 10km to 50km
df['maximum_distance'] = np.random.randint(10, 51, size=len(df))

# Save the updated file back to Google Drive
file_path = "/content/drive/My Drive/DataPre/preprocessed_user_inputs.csv"
df.to_csv(file_path, index=False)

print("File successfully saved!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File successfully saved!


In [43]:
# Load the Excel file
file_path = '/content/drive/My Drive/Colombo/RestaurantsColombo.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the data
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,address,addressObj/city,addressObj/country,addressObj/postalcode,addressObj/street1,addressObj/street2,ancestorLocations/0/id,ancestorLocations/0/name,ancestorLocations/0/subcategory,ancestorLocations/1/id,...,reviewTags/27/text,reviewTags/28/reviews,reviewTags/28/text,reviewTags/29/reviews,reviewTags/29/text,subcategories/0,travelerChoiceAward,type,webUrl,website
0,"57 Ward Place Roof Top, Jetwing Colombo Seven,...",Colombo,Sri Lanka,700.0,57 Ward Place,"Roof Top, Jetwing Colombo Seven",293962,Colombo,City,2467812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,https://www.jetwinghotels.com/jetwingcolombose...
1,"2 Galle Road, Colombo 00300 Sri Lanka",Colombo,Sri Lanka,300.0,2 Galle Road,NaN,293962,Colombo,City,2467812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,https://www.facebook.com/1864restaurant/
2,"106 Thimbirigasyaya Road, Colombo 00500 Sri Lanka",Colombo,Sri Lanka,500.0,106 Thimbirigasyaya Road,NaN,293962,Colombo,City,2467812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.facebook.com/bubblemebubbletea
3,"10 Galle Face Drive, Colombo 00300 Sri Lanka",Colombo,Sri Lanka,300.0,10 Galle Face Drive,NaN,293962,Colombo,City,2467812,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://barracuda.lk
4,"590 Colombo - Galle Main Road Marino Mall, Col...",Colombo,Sri Lanka,NaN,590 Colombo - Galle Main Road,Marino Mall,293962,Colombo,City,2467812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,NaN


In [45]:
# Renaming Columns for Consistency: Replace '/' with '_'
# This step standardizes column names by replacing '/' with '_' to avoid issues in processing
df.columns = [col.strip().lower().replace(' ', '_').replace('/', '_') for col in df.columns]

# Append 'addressobj_postalcode' to 'address', separated by a comma, if the column exists
# This step checks if 'addressobj_postalcode' is in the dataframe
if 'addressobj_postalcode' in df.columns:
    # Combine 'address' and 'addressobj_postalcode', separating them with a comma if both are non-empty
    df['address'] = df.apply(
        lambda row: f"{row['address']}, {row['addressobj_postalcode']}" if pd.notna(row['addressobj_postalcode']) else row['address'], axis=1)
    # Drop the 'addressobj_postalcode' column after appending its data
    df = df.drop(columns=['addressobj_postalcode'])

# Combine 'addressobj_street2' into 'addressobj_street1' and rename it to 'addressobj_street'
# This step checks if 'addressobj_street2' is in the dataframe
if 'addressobj_street2' in df.columns and 'addressobj_street1' in df.columns:
    # Combine 'addressobj_street1' and 'addressobj_street2', separated by a comma if both are non-empty
    df['addressobj_street1'] = df.apply(
        lambda row: f"{row['addressobj_street1']}, {row['addressobj_street2']}" if pd.notna(row['addressobj_street2']) else row['addressobj_street1'], axis=1)
    # Rename 'addressobj_street1' to 'addressobj_street'
    df.rename(columns={'addressobj_street1': 'addressobj_street'}, inplace=True)
    # Drop the 'addressobj_street2' column after appending its data
    df = df.drop(columns=['addressobj_street2'])

# Function to combine multiple columns into one
def combine_columns(row, column_list):
    # Check if the column_list exists and has valid columns
    valid_columns = [col for col in column_list if col in row.index]

    if valid_columns:
        combined_value = ', '.join(row[valid_columns].dropna().astype(str)).strip()
        if combined_value:
            return combined_value

    return "unknown"

# Identify and merge grouped columns
categories = ['cuisines', 'dietaryrestrictions', 'dishes', 'features', 'mealtypes', 'establishmenttypes']
for category in categories:
    category_columns = [col for col in df.columns if col.startswith(f'{category}_')]

    if category_columns:
        df.loc[:, category] = df.apply(lambda row: combine_columns(row, category_columns), axis=1)
        df = df.drop(columns=category_columns, errors='ignore')  # `errors='ignore'` ensures safe column dropping

# Drop unnecessary columns
drop_columns = [col for col in df.columns if col.startswith(('photos_', 'hours_', 'orderonline_', 'ownerstopreasons', 'price_', 'ancestorlocations_',
                                                             'nearestmetrostations_', 'neighborhoodlocations_', 'addressobj_', 'ratinghistogram_', 'reviewtags_'))]
df = df.drop(columns=drop_columns, errors='ignore')

# These columns are no longer needed in the dataset
columns_to_drop = ['hours', 'input', 'isclaimedicon', 'isclaimedtext', 'isclosed', 'islongclosed', 'image', 'isnearbyresult', 'addressobj_state', 'travelerchoiceaward', 'establishmenttypes', 'dishes', 'type', 'website',
                   'isnearbyresult', 'localaddress', 'localname', 'opennowtext', 'photocount', 'pricerange', 'locallangcode', 'description', 'email', 'id', 'locationstring','menuweburl', 'subcategories_0', 'phone'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Replace missing values with the appropriate message
def replace_missing_values(row, col):
    if pd.isna(row[col]):
        return "unknown"
    else:
        return row[col]

# Apply default message logic for missing values
default_message_columns = ['pricelevel', 'website']
for col in default_message_columns:
    if col in df.columns:
        df[col] = df.apply(lambda row: replace_missing_values(row, col), axis=1)

columns_to_replace = [
    'latitude', 'longitude', 'travelerchoiceaward', 'rankingdenominator',
    'rankingposition', 'rankingstring', 'rating', 'rawranking', 'reviews_and_text'
]

# Replace missing values: text columns with 'unknown', numeric score columns with 0
for col in columns_to_replace:
    if col in df.columns:
      df[col] = df[col].fillna('unknown')

# Print final columns to verify
print(df.columns)
data = df.head()
display(data)

# Remove duplicate rows based on all columns
df.drop_duplicates()

# Save the modified data to a new Excel file
# The preprocessed file contains standardized column names, a combined 'amenities' column,
# an updated 'address' column, and an updated 'addressobj_street' column
output_file = '/content/drive/My Drive/Colombo/PreprocessedRestaurantsColombo.csv'
df.to_csv(output_file, index=False)

print(f"Preprocessing complete. Cleaned data saved to '{output_file}'")

Index(['address', 'category', 'latitude', 'longitude', 'name',
       'numberofreviews', 'pricelevel', 'rankingdenominator',
       'rankingposition', 'rankingstring', 'rating', 'rawranking', 'weburl',
       'cuisines', 'dietaryrestrictions', 'features', 'mealtypes'],
      dtype='object')


,address,category,latitude,longitude,name,numberofreviews,pricelevel,rankingdenominator,rankingposition,rankingstring,rating,rawranking,weburl,cuisines,dietaryrestrictions,features,mealtypes
0,"57 Ward Place Roof Top, Jetwing Colombo Seven,...",restaurant,6.91658,79.86995,Ward 7,42,$$ - $$$,880,148,#148 of 880 Restaurants in Colombo,4.5,3.417001,https://www.tripadvisor.com/Restaurant_Review-...,"Bar, Pub",unknown,"Reservations, Outdoor Seating, Seating, Televi...","Lunch, Dinner, Brunch, Late Night, Drinks"
1,"2 Galle Road, Colombo 00300 Sri Lanka, 300.0",restaurant,unknown,unknown,1864 Limited Edition,18,$$$$,880,131,#131 of 880 Restaurants in Colombo,5.0,3.457108,https://www.tripadvisor.com/Restaurant_Review-...,French,unknown,"Reservations, Private Dining, Seating, Parking...","Dinner, Late Night, Drinks"
2,"106 Thimbirigasyaya Road, Colombo 00500 Sri La...",restaurant,6.89345,79.87038,Bubble Me Bubble Tea,26,$$ - $$$,880,276,#276 of 880 Restaurants in Colombo,4.0,3.165838,https://www.tripadvisor.com/Restaurant_Review-...,Cafe,unknown,Takeout,unknown
3,"10 Galle Face Drive, Colombo 00300 Sri Lanka, ...",restaurant,6.921183,79.84617,Breeze Barracuda,104,$$ - $$$,880,196,#196 of 880 Restaurants in Colombo,4.0,3.286445,https://www.tripadvisor.com/Restaurant_Review-...,"Asian, Contemporary, Sri Lankan",Vegetarian friendly,"Reservations, Outdoor Seating, Seating, Serves...","Lunch, Dinner"
4,"590 Colombo - Galle Main Road Marino Mall, Col...",restaurant,6.900297,79.85292,Shiwu Chinese Restaurant,41,$$ - $$$,800,233,"#233 of 1,135 places to eat in Colombo",4.0,3.273561,https://www.tripadvisor.com/Restaurant_Review-...,"Chinese, Asian","Vegetarian friendly, Vegan options","Reservations, Seating, Serves Alcohol, Table S...","Lunch, Dinner"


Preprocessing complete. Cleaned data saved to '/content/drive/My Drive/Colombo/PreprocessedRestaurantsColombo.csv'


In [46]:
# Load the Excel file
file_path = '/content/drive/My Drive/Ella/RestaurantsElla.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the data
df.head()

,address,addressObj/city,addressObj/country,addressObj/postalcode,addressObj/state,addressObj/street1,addressObj/street2,ancestorLocations/0/abbreviation,ancestorLocations/0/id,ancestorLocations/0/name,...,reviewTags/27/text,reviewTags/28/reviews,reviewTags/28/text,reviewTags/29/reviews,reviewTags/29/text,subcategories/0,travelerChoiceAward,type,webUrl,website
0,"Wellawaya Road, Ella Ella's Edge Resort, Ella ...",Ella,Sri Lanka,90090.0,NaN,"Wellawaya Road, Ella",Ella's Edge Resort,NaN,616035,Ella,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.ellasedgeresort.com
1,"No 56 Wellawaya Rd, Ella 90090 Sri Lanka",Ella,Sri Lanka,90090.0,NaN,No 56 Wellawaya Rd,NaN,NaN,616035,Ella,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.facebook.com/CafeCElla
2,"Ella Rd, Ella 90090 Sri Lanka",Ella,Sri Lanka,90090.0,NaN,Ella Rd,NaN,NaN,616035,Ella,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.laellabreeze.com
3,"Railway Station Road, Ella 90090 Sri Lanka",Ella,Sri Lanka,90090.0,NaN,Railway Station Road,NaN,NaN,616035,Ella,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.ellalovers.com
4,"No. 36 Waterfall Road, off Ella/Wellawaya Road...",Ella,Sri Lanka,90090.0,NaN,"No. 36 Waterfall Road, off Ella/Wellawaya Road",NaN,NaN,616035,Ella,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.ellaspirit.org


In [47]:
# Renaming Columns for Consistency: Replace '/' with '_'
# This step standardizes column names by replacing '/' with '_' to avoid issues in processing
df.columns = [col.strip().lower().replace(' ', '_').replace('/', '_') for col in df.columns]

# Append 'addressobj_postalcode' to 'address', separated by a comma, if the column exists
# This step checks if 'addressobj_postalcode' is in the dataframe
if 'addressobj_postalcode' in df.columns:
    # Combine 'address' and 'addressobj_postalcode', separating them with a comma if both are non-empty
    df['address'] = df.apply(
        lambda row: f"{row['address']}, {row['addressobj_postalcode']}" if pd.notna(row['addressobj_postalcode']) else row['address'], axis=1)
    # Drop the 'addressobj_postalcode' column after appending its data
    df = df.drop(columns=['addressobj_postalcode'])

# Combine 'addressobj_street2' into 'addressobj_street1' and rename it to 'addressobj_street'
# This step checks if 'addressobj_street2' is in the dataframe
if 'addressobj_street2' in df.columns and 'addressobj_street1' in df.columns:
    # Combine 'addressobj_street1' and 'addressobj_street2', separated by a comma if both are non-empty
    df['addressobj_street1'] = df.apply(
        lambda row: f"{row['addressobj_street1']}, {row['addressobj_street2']}" if pd.notna(row['addressobj_street2']) else row['addressobj_street1'], axis=1)
    # Rename 'addressobj_street1' to 'addressobj_street'
    df.rename(columns={'addressobj_street1': 'addressobj_street'}, inplace=True)
    # Drop the 'addressobj_street2' column after appending its data
    df = df.drop(columns=['addressobj_street2'])

# Function to combine multiple columns into one
def combine_columns(row, column_list):
    # Check if the column_list exists and has valid columns
    valid_columns = [col for col in column_list if col in row.index]

    if valid_columns:
        combined_value = ', '.join(row[valid_columns].dropna().astype(str)).strip()
        if combined_value:
            return combined_value

    return "unknown"

# Identify and merge grouped columns
categories = ['cuisines', 'dietaryrestrictions', 'dishes', 'features', 'mealtypes', 'establishmenttypes']
for category in categories:
    category_columns = [col for col in df.columns if col.startswith(f'{category}_')]

    if category_columns:
        df.loc[:, category] = df.apply(lambda row: combine_columns(row, category_columns), axis=1)
        df = df.drop(columns=category_columns, errors='ignore')  # `errors='ignore'` ensures safe column dropping

# Drop unnecessary columns
drop_columns = [col for col in df.columns if col.startswith(('photos_', 'hours_', 'orderonline_', 'ownerstopreasons', 'price_', 'ancestorlocations_', 'nearestmetrostations_', 'neighborhoodlocations_', 'addressobj_', 'ratinghistogram_', 'reviewtags_'))]
df = df.drop(columns=drop_columns, errors='ignore')

# These columns are no longer needed in the dataset
columns_to_drop = ['hours', 'input', 'isclaimedicon', 'isclaimedtext', 'isclosed', 'islongclosed', 'image', 'isnearbyresult', 'addressobj_state', 'travelerchoiceaward', 'establishmenttypes', 'dishes', 'type', 'website',
                   'isnearbyresult', 'localaddress', 'localname', 'opennowtext', 'photocount', 'pricerange', 'locallangcode', 'description', 'email', 'id', 'locationstring','menuweburl', 'subcategories_0', 'phone'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Replace missing values with the appropriate message
def replace_missing_values(row, col):
    if pd.isna(row[col]):
        return f"unknown"
    else:
        return row[col]

# Apply default message logic for missing values
default_message_columns = ['pricelevel', 'website']
for col in default_message_columns:
    if col in df.columns:
        df[col] = df.apply(lambda row: replace_missing_values(row, col), axis=1)

columns_to_replace = [
    'latitude', 'longitude', 'travelerchoiceaward', 'rankingdenominator',
    'rankingposition', 'rankingstring', 'rating', 'rawranking', 'reviews_and_text'
]

# Replace missing values: text columns with 'not provided', numeric score columns with 0
for col in columns_to_replace:
    if col in df.columns:
      df[col] = df[col].fillna('unknown')

# Print final columns to verify
print(df.columns)
data = df.head()
display(data)

# Save the modified data to a new Excel file
# The preprocessed file contains standardized column names, a combined 'amenities' column,
# an updated 'address' column, and an updated 'addressobj_street' column
output_file = '/content/drive/My Drive/Ella/PreprocessedRestaurantsElla.csv'
df.to_csv(output_file, index=False)

print(f"Preprocessing complete. Cleaned data saved to '{output_file}'")

Index(['address', 'category', 'latitude', 'longitude', 'name',
       'numberofreviews', 'pricelevel', 'rankingdenominator',
       'rankingposition', 'rankingstring', 'rating', 'rawranking', 'weburl',
       'cuisines', 'dietaryrestrictions', 'features', 'mealtypes'],
      dtype='object')


,address,category,latitude,longitude,name,numberofreviews,pricelevel,rankingdenominator,rankingposition,rankingstring,rating,rawranking,weburl,cuisines,dietaryrestrictions,features,mealtypes
0,"Wellawaya Road, Ella Ella's Edge Resort, Ella ...",restaurant,6.870678,81.04761,Ella's Edge Restaurant,44,$$ - $$$,177.0,72.0,#72 of 177 Restaurants in Ella,4.5,3.306603,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, Chinese, Indian, Asian, Sri Lankan",Vegetarian friendly,"Outdoor Seating, Seating, Table Service","Breakfast, Lunch, Dinner"
1,"No 56 Wellawaya Rd, Ella 90090 Sri Lanka, 90090.0",restaurant,6.874613,81.04735,Cafe C Ella,435,$,154.0,19.0,#19 of 203 places to eat in Ella,5.0,4.67991,https://www.tripadvisor.com/Restaurant_Review-...,"Chinese, Grill, Street Food, Sri Lankan","Vegetarian friendly, Vegan options, Gluten fre...","Takeout, Outdoor Seating, Seating, Wheelchair ...","Lunch, Dinner"
2,"Ella Rd, Ella 90090 Sri Lanka, 90090.0",restaurant,6.873278,81.04905,Oak Ray La Ella Breeze,631,$,177.0,53.0,#53 of 177 Restaurants in Ella,4.0,3.697622,https://www.tripadvisor.com/Restaurant_Review-...,"Beijing Specialties, NorthWestern Chinese, Sri...","Vegetarian friendly, Vegan options","Takeout, Reservations, Outdoor Seating, Buffet...","Breakfast, Lunch, Dinner, Late Night"
3,"Railway Station Road, Ella 90090 Sri Lanka, 90...",restaurant,6.874612,81.046936,Hiking Bar & Restaurant,885,$,177.0,9.0,#9 of 177 Restaurants in Ella,5.0,4.829502,https://www.tripadvisor.com/Restaurant_Review-...,"Chinese, Bar, Cafe, Asian, Sri Lankan","Vegetarian friendly, Vegan options","Takeout, Outdoor Seating, Seating, Street Park...","Breakfast, Lunch, Dinner, Brunch, Drinks"
4,"No. 36 Waterfall Road, off Ella/Wellawaya Road...",restaurant,6.825725,81.07735,BarCeylona Space Cafe,1,$$ - $$$,177.0,138.0,#138 of 177 Restaurants in Ella,5.0,3.019732,https://www.tripadvisor.com/Restaurant_Review-...,"French, German, Asian, Sri Lankan",unknown,unknown,unknown


Preprocessing complete. Cleaned data saved to '/content/drive/My Drive/Ella/PreprocessedRestaurantsElla.csv'


In [48]:
# Load the Excel file
file_path = '/content/drive/My Drive/Kandy/KandyRestaurants.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the data
df.head()

,address,addressObj/city,addressObj/country,addressObj/postalcode,addressObj/state,addressObj/street1,addressObj/street2,ancestorLocations/0/abbreviation,ancestorLocations/0/id,ancestorLocations/0/name,...,reviewTags/25/text,reviewTags/26/reviews,reviewTags/26/text,reviewTags/27/reviews,reviewTags/27/text,subcategories/0,travelerChoiceAward,type,webUrl,website
0,"12 Mahamaya Mawatha, Kandy 20000 Sri Lanka",Kandy,Sri Lanka,20000.0,NaN,12 Mahamaya Mawatha,NaN,NaN,304138,Kandy,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,https://www.grandserendibhotel.com/index.php
1,"839 Peradeniya Road Kandyan Arts Residency, Ka...",Kandy,Sri Lanka,20000.0,NaN,839 Peradeniya Road,Kandyan Arts Residency,NaN,304138,Kandy,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://hotelkandyanarts.com/
2,"9 Sangaraja Mawatha, Kandy 8 Sri Lanka",Kandy,Sri Lanka,8.0,NaN,9 Sangaraja Mawatha,NaN,NaN,304138,Kandy,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,NaN
3,"155 S W R D Bandaranayake Road, Kandy 20000 Sr...",Kandy,Sri Lanka,20000.0,NaN,155 S W R D Bandaranayake Road,NaN,NaN,304138,Kandy,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.kandycitystay.com
4,"No. 1C Devi Road Galkaduwa Junction, Kandy 200...",Kandy,Sri Lanka,20000.0,NaN,No. 1C Devi Road,Galkaduwa Junction,NaN,304138,Kandy,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,https://www.facebook.com/profile.php?id=615576...


In [49]:
# Renaming Columns for Consistency: Replace '/' with '_'
# This step standardizes column names by replacing '/' with '_' to avoid issues in processing
df.columns = [col.strip().lower().replace(' ', '_').replace('/', '_') for col in df.columns]

# Append 'addressobj_postalcode' to 'address', separated by a comma, if the column exists
# This step checks if 'addressobj_postalcode' is in the dataframe
if 'addressobj_postalcode' in df.columns:
    # Combine 'address' and 'addressobj_postalcode', separating them with a comma if both are non-empty
    df['address'] = df.apply(
        lambda row: f"{row['address']}, {row['addressobj_postalcode']}" if pd.notna(row['addressobj_postalcode']) else row['address'], axis=1)
    # Drop the 'addressobj_postalcode' column after appending its data
    df = df.drop(columns=['addressobj_postalcode'])

# Combine 'addressobj_street2' into 'addressobj_street1' and rename it to 'addressobj_street'
# This step checks if 'addressobj_street2' is in the dataframe
if 'addressobj_street2' in df.columns and 'addressobj_street1' in df.columns:
    # Combine 'addressobj_street1' and 'addressobj_street2', separated by a comma if both are non-empty
    df['addressobj_street1'] = df.apply(
        lambda row: f"{row['addressobj_street1']}, {row['addressobj_street2']}" if pd.notna(row['addressobj_street2']) else row['addressobj_street1'], axis=1)
    # Rename 'addressobj_street1' to 'addressobj_street'
    df.rename(columns={'addressobj_street1': 'addressobj_street'}, inplace=True)
    # Drop the 'addressobj_street2' column after appending its data
    df = df.drop(columns=['addressobj_street2'])

# Function to combine multiple columns into one
def combine_columns(row, column_list):
    # Check if the column_list exists and has valid columns
    valid_columns = [col for col in column_list if col in row.index]

    if valid_columns:
        combined_value = ', '.join(row[valid_columns].dropna().astype(str)).strip()
        if combined_value:
            return combined_value

    return "unknown"

# Identify and merge grouped columns
categories = ['cuisines', 'dietaryrestrictions', 'dishes', 'features', 'mealtypes', 'establishmenttypes']
for category in categories:
    category_columns = [col for col in df.columns if col.startswith(f'{category}_')]

    if category_columns:
        df.loc[:, category] = df.apply(lambda row: combine_columns(row, category_columns), axis=1)
        df = df.drop(columns=category_columns, errors='ignore')  # `errors='ignore'` ensures safe column dropping

# Drop unnecessary columns
drop_columns = [col for col in df.columns if col.startswith(('photos_', 'hours_', 'orderonline_', 'ownerstopreasons', 'price_', 'ancestorlocations_', 'nearestmetrostations_', 'addressobj_', 'ratinghistogram_', 'reviewtags_'))]
df = df.drop(columns=drop_columns, errors='ignore')

# These columns are no longer needed in the dataset
columns_to_drop = ['hours', 'input', 'isclaimedicon', 'isclaimedtext', 'isclosed', 'islongclosed', 'image', 'isnearbyresult', 'addressobj_state', 'travelerchoiceaward', 'establishmenttypes', 'dishes',  'type', 'website',
                   'isnearbyresult', 'localaddress', 'localname', 'opennowtext', 'photocount', 'pricerange', 'locallangcode', 'description', 'email', 'id', 'locationstring','menuweburl', 'subcategories_0', 'phone'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Replace missing values with the appropriate message
def replace_missing_values(row, col):
    if pd.isna(row[col]):
        return "unknown"
    else:
        return row[col]

# Apply default message logic for missing values
default_message_columns = ['pricelevel', 'website']
for col in default_message_columns:
    if col in df.columns:
        df[col] = df.apply(lambda row: replace_missing_values(row, col), axis=1)

columns_to_replace = [
    'latitude', 'longitude', 'rankingdenominator',
    'rankingposition', 'rankingstring', 'rating', 'rawranking'
]

# Replace missing values: text columns with 'unknown', numeric score columns with 0
for col in columns_to_replace:
    if col in df.columns:
      df[col] = df[col].fillna('unknown')

# Print final columns to verify
print(df.columns)
data = df.head()
display(data)

# Save the modified data to a new Excel file
# The preprocessed file contains standardized column names, a combined 'amenities' column,
# an updated 'address' column, and an updated 'addressobj_street' column
output_file = '/content/drive/My Drive/Kandy/PreprocessedRestaurantsKandy.csv'
df.to_csv(output_file, index=False)

print(f"Preprocessing complete. Cleaned data saved to '{output_file}'")

Index(['address', 'category', 'latitude', 'longitude', 'name',
       'numberofreviews', 'pricelevel', 'rankingdenominator',
       'rankingposition', 'rankingstring', 'rating', 'rawranking', 'weburl',
       'cuisines', 'dietaryrestrictions', 'features', 'mealtypes'],
      dtype='object')


,address,category,latitude,longitude,name,numberofreviews,pricelevel,rankingdenominator,rankingposition,rankingstring,rating,rawranking,weburl,cuisines,dietaryrestrictions,features,mealtypes
0,"12 Mahamaya Mawatha, Kandy 20000 Sri Lanka, 20...",restaurant,7.285323,80.644485,Grand Sky Lounge,64,$$$$,321,3,#3 of 321 Restaurants in Kandy,5.0,4.683845,https://www.tripadvisor.com/Restaurant_Review-...,"Bar, International, Pub, Dining bars",unknown,"Reservations, Outdoor Seating, Seating, Parkin...","Breakfast, Lunch, Dinner, Brunch, Drinks"
1,"839 Peradeniya Road Kandyan Arts Residency, Ka...",restaurant,7.276945,80.615524,Kandyan Arts Restaurant,238,$$ - $$$,321,48,#48 of 321 Restaurants in Kandy,3.5,3.443559,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, International, Asian, Indonesian, Mon...","Vegetarian friendly, Vegan options","Reservations, Outdoor Seating, Buffet, Seating...","Lunch, Dinner, Drinks"
2,"9 Sangaraja Mawatha, Kandy 8 Sri Lanka, 8.0",restaurant,7.286927,80.64747,The Kandy Garden Cafe,683,$,321,17,#17 of 321 Restaurants in Kandy,4.0,4.077689,https://www.tripadvisor.com/Restaurant_Review-...,"Asian, Sri Lankan","Vegetarian friendly, Vegan options","Takeout, Seating, Table Service","Breakfast, Lunch, Dinner, Late Night"
3,"155 S W R D Bandaranayake Road, Kandy 20000 Sr...",restaurant,7.286809,80.627655,The Whizz,102,$$ - $$$,321,41,#41 of 321 Restaurants in Kandy,4.5,3.499032,https://www.tripadvisor.com/Restaurant_Review-...,Cafe,"Vegetarian friendly, Vegan options","Reservations, Outdoor Seating, Seating, Parkin...","Breakfast, Lunch, Dinner, Late Night, Drinks"
4,"No. 1C Devi Road Galkaduwa Junction, Kandy 200...",restaurant,7.2955,80.6356,The Chef Corner & Vip Lounge By Sugath,6,$$ - $$$,321,89,#89 of 321 Restaurants in Kandy,5.0,3.194740,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, Chinese, Asian, Sri Lankan",unknown,"Delivery, Takeout, Seating, Parking Available,...","Breakfast, Lunch, Dinner, Brunch, Drinks"


Preprocessing complete. Cleaned data saved to '/content/drive/My Drive/Kandy/PreprocessedRestaurantsKandy.csv'


In [50]:
# Load the Excel file
file_path = '/content/drive/My Drive/NuwaraEliya/RestaurantsNuwaraEliya.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the data
df.head()

,address,addressObj/city,addressObj/country,addressObj/postalcode,addressObj/state,addressObj/street1,addressObj/street2,ancestorLocations/0/abbreviation,ancestorLocations/0/id,ancestorLocations/0/name,...,reviewTags/19/text,reviewTags/20/reviews,reviewTags/20/text,reviewTags/21/reviews,reviewTags/21/text,subcategories/0,travelerChoiceAward,type,webUrl,website
0,"18 Badulla Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,22200.0,NaN,18 Badulla Road,NaN,NaN,608524,Nuwara Eliya,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,NaN
1,"158, Kandy Road, Nuwara Eliya 22220 Sri Lanka",Nuwara Eliya,Sri Lanka,22220.0,NaN,"158, Kandy Road",NaN,NaN,608524,Nuwara Eliya,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.pizzahut.lk
2,"20 Kandy Road, Nuwara Eliya 22200 Sri Lanka",Nuwara Eliya,Sri Lanka,22200.0,NaN,20 Kandy Road,NaN,NaN,608524,Nuwara Eliya,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.facebook.com/The-Pub-407674586680194/
3,"405 Kandy Road Bambarakale, Nuwaraeliya, Nuwar...",Nuwara Eliya,Sri Lanka,22200.0,NaN,"405 Kandy Road Bambarakale, Nuwaraeliya",NaN,NaN,608524,Nuwara Eliya,...,NaN,NaN,NaN,NaN,NaN,Sit down,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,http://www.midkyhotels.com
4,"Nuwaraeliya Road, Talawakele 22100 Sri Lanka",Talawakele,Sri Lanka,22100.0,NaN,Nuwaraeliya Road,NaN,NaN,12158457,Talawakele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RESTAURANT,https://www.tripadvisor.com/Restaurant_Review-...,NaN


In [51]:
# Renaming Columns for Consistency: Replace '/' with '_'
# This step standardizes column names by replacing '/' with '_' to avoid issues in processing
df.columns = [col.strip().lower().replace(' ', '_').replace('/', '_') for col in df.columns]

# Append 'addressobj_postalcode' to 'address', separated by a comma, if the column exists
# This step checks if 'addressobj_postalcode' is in the dataframe
if 'addressobj_postalcode' in df.columns:
    # Combine 'address' and 'addressobj_postalcode', separating them with a comma if both are non-empty
    df['address'] = df.apply(
        lambda row: f"{row['address']}, {row['addressobj_postalcode']}" if pd.notna(row['addressobj_postalcode']) else row['address'], axis=1)
    # Drop the 'addressobj_postalcode' column after appending its data
    df = df.drop(columns=['addressobj_postalcode'])

# Combine 'addressobj_street2' into 'addressobj_street1' and rename it to 'addressobj_street'
# This step checks if 'addressobj_street2' is in the dataframe
if 'addressobj_street2' in df.columns and 'addressobj_street1' in df.columns:
    # Combine 'addressobj_street1' and 'addressobj_street2', separated by a comma if both are non-empty
    df['addressobj_street1'] = df.apply(
        lambda row: f"{row['addressobj_street1']}, {row['addressobj_street2']}" if pd.notna(row['addressobj_street2']) else row['addressobj_street1'], axis=1)
    # Rename 'addressobj_street1' to 'addressobj_street'
    df.rename(columns={'addressobj_street1': 'addressobj_street'}, inplace=True)
    # Drop the 'addressobj_street2' column after appending its data
    df = df.drop(columns=['addressobj_street2'])

# Function to combine multiple columns into one
def combine_columns(row, column_list):
    # Check if the column_list exists and has valid columns
    valid_columns = [col for col in column_list if col in row.index]

    if valid_columns:
        combined_value = ', '.join(row[valid_columns].dropna().astype(str)).strip()
        if combined_value:
            return combined_value

    return "unknown"

# Identify and merge grouped columns
categories = ['cuisines', 'dietaryrestrictions', 'dishes', 'features', 'mealtypes', 'establishmenttypes']
for category in categories:
    category_columns = [col for col in df.columns if col.startswith(f'{category}_')]

    if category_columns:
        df.loc[:, category] = df.apply(lambda row: combine_columns(row, category_columns), axis=1)
        df = df.drop(columns=category_columns, errors='ignore')  # `errors='ignore'` ensures safe column dropping

# Drop unnecessary columns
drop_columns = [col for col in df.columns if col.startswith(('photos_', 'hours_', 'orderonline_', 'ownerstopreasons', 'price_', 'ancestorlocations_', 'addressobj_', 'ratinghistogram_', 'reviewtags_'))]
df = df.drop(columns=drop_columns, errors='ignore')

# These columns are no longer needed in the dataset
columns_to_drop = ['hours', 'input', 'isclaimedicon', 'isclaimedtext', 'isclosed', 'islongclosed', 'image', 'isnearbyresult', 'addressobj_state', 'travelerchoiceaward', 'establishmenttypes', 'dishes',  'type', 'website',
                   'isnearbyresult', 'localaddress', 'localname', 'opennowtext', 'photocount', 'pricerange', 'description', 'email', 'id', 'locationstring','menuweburl', 'subcategories_0', 'phone'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Replace missing values with the appropriate message
def replace_missing_values(row, col):
    if pd.isna(row[col]):
        return "unknown"
    else:
        return row[col]

# Apply default message logic for missing values
default_message_columns = ['pricelevel', 'website']
for col in default_message_columns:
    if col in df.columns:
        df[col] = df.apply(lambda row: replace_missing_values(row, col), axis=1)

columns_to_replace = [
    'latitude', 'longitude', 'travelerchoiceaward', 'rankingdenominator',
    'rankingposition', 'rankingstring', 'rating', 'rawranking'
]

# Replace missing values: text columns with 'not provided', numeric score columns with 0
for col in columns_to_replace:
    if col in df.columns:
      df[col] = df[col].fillna('unknown')

# Print final columns to verify
print(df.columns)
data = df.head()
display(data)

# Save the modified data to a new Excel file
# The preprocessed file contains standardized column names, a combined 'amenities' column,
# an updated 'address' column, and an updated 'addressobj_street' column
output_file = '/content/drive/My Drive/NuwaraEliya/PreprocessedRestaurantsNuwaraEliya.csv'
df.to_csv(output_file, index=False)

print(f"Preprocessing complete. Cleaned data saved to '{output_file}'")

Index(['address', 'category', 'latitude', 'longitude', 'name',
       'numberofreviews', 'pricelevel', 'rankingdenominator',
       'rankingposition', 'rankingstring', 'rating', 'rawranking', 'weburl',
       'cuisines', 'dietaryrestrictions', 'features', 'mealtypes'],
      dtype='object')


,address,category,latitude,longitude,name,numberofreviews,pricelevel,rankingdenominator,rankingposition,rankingstring,rating,rawranking,weburl,cuisines,dietaryrestrictions,features,mealtypes
0,"18 Badulla Road, Nuwara Eliya 22200 Sri Lanka,...",restaurant,6.956621,80.77682,Indian Summer Restaurant,4,$$ - $$$,87.0,59.0,#59 of 116 places to eat in Nuwara Eliya,4.0,3.067908,https://www.tripadvisor.com/Restaurant_Review-...,unknown,unknown,unknown,"Lunch, Dinner"
1,"158, Kandy Road, Nuwara Eliya 22220 Sri Lanka,...",restaurant,6.97488,80.76336,Pizza Hut,28,$$ - $$$,101.0,72.0,#72 of 101 Restaurants in Nuwara Eliya,3.0,3.021859,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, Pizza, Fast Food",Vegetarian friendly,"Delivery, Takeout",unknown
2,"20 Kandy Road, Nuwara Eliya 22200 Sri Lanka, 2...",restaurant,6.974394,80.76689,The Pub,159,$$ - $$$,87.0,56.0,#56 of 116 places to eat in Nuwara Eliya,3.0,3.074628,https://www.tripadvisor.com/Restaurant_Review-...,"Bar, Pub",unknown,"Takeout, Seating, Television, Serves Alcohol, ...","Lunch, Dinner"
3,"405 Kandy Road Bambarakale, Nuwaraeliya, Nuwar...",restaurant,6.97809,80.75889,Midky Restaurant,0,$,unknown,unknown,unknown,unknown,unknown,https://www.tripadvisor.com/Restaurant_Review-...,"Italian, Chinese, Indian, Asian, Sri Lankan, A...",unknown,"Takeout, Reservations, Outdoor Seating, Seatin...","Breakfast, Lunch, Dinner"
4,"Nuwaraeliya Road, Talawakele 22100 Sri Lanka, ...",restaurant,6.933452,80.66997,Kochchi Family Restaurant,0,unknown,unknown,unknown,unknown,unknown,unknown,https://www.tripadvisor.com/Restaurant_Review-...,"Asian, Sri Lankan",unknown,unknown,unknown


Preprocessing complete. Cleaned data saved to '/content/drive/My Drive/NuwaraEliya/PreprocessedRestaurantsNuwaraEliya.csv'


In [52]:
import pandas as pd
import os
import glob

# List of folder paths where the CSVs are stored
folders = [
    "/content/drive/My Drive/NuwaraEliya",
    "/content/drive/My Drive/Ella",
    "/content/drive/My Drive/Kandy",
    "/content/drive/My Drive/Colombo"
]

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over each folder and read only CSV files that contain "Preprocessed" in their filename
for folder in folders:
    csv_files = glob.glob(os.path.join(folder, "PreprocessedRestaurants*.csv"))  # Filter files with "Preprocessed" in name

    for file in csv_files:
        df = pd.read_csv(file)  # Read CSV
        dataframes.append(df)   # Append DataFrame to the list

# Concatenate all DataFrames into a single one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame as a CSV
merged_df.to_csv("/content/drive/My Drive/Colombo/PreprocessedMergedRestaurants.csv", index=False)

print("Merged CSV saved as merged_output.csv")

Merged CSV saved as merged_output.csv
